In [393]:
#import libraries

import  pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import math

## 1.- Calculate profit margin 

In [394]:
## import "Sales line Item Table" data set
sales_df = pd.read_excel("Planning Manager - Test.xlsx", sheet_name="Sales line Item Table")
sales_df

,Customer Number,Customer Name,Order Number,Sku#,Version,Qty. sold,Unit Price (cost per unit),posted date
0,C012345,Luis,123,2970,V11,50,10,2022-03-10
1,C012345,Luis,123,2974,V12,50,10,2022-03-10
2,C012345,Luis,123,2978,NaN,50,10,2022-03-10
3,C012345,Luis,123,3004,V1,50,10,2022-03-10
4,C012345,Luis,123,3005,V2,50,10,2022-03-10
...,...,...,...,...,...,...,...,...
85,C012347,Ralph,125,2970,V11,30,10,2021-10-05
86,C012347,Ralph,125,2974,V12,30,10,2021-10-05
87,C012347,Ralph,125,2978,NaN,30,10,2021-10-05
88,C012347,Ralph,125,3004,V1,30,10,2021-10-05


In [395]:
## import "Item Table" data set
items_df = pd.read_excel("Planning Manager - Test.xlsx", sheet_name="Item Table")
items_df

,Sku#,Description,UOM,Unit Cost,Active Version
0,2970,RL - Revitabrow Advanced 3.0,EA,1.25,V11
1,2974,RL- Revitalash 2.0ml,EA,1.10,V12
2,2978,RL- Revitalash 1.0ml,EA,2.30,NaN
3,3004,RL - Brow Gel Soft Brown,EA,1.20,V1
4,3005,RL - Brow Gel Dark Brown,EA,1.30,V2


In [396]:
## import "Inventory table" data set
inv_df = pd.read_excel("Planning Manager - Test.xlsx", sheet_name="Inventory table")
inv_df

,Sku#,Description,Version,Qty. on hand
0,2970,RL - Revitabrow Advanced 3.0,V11,210
1,2974,RL- Revitalash 2.0ml,V12,210
2,2978,RL- Revitalash 1.0ml,NaN,210
3,3004,RL - Brow Gel Soft Brown,V1,210
4,3005,RL - Brow Gel Dark Brown,V2,210


In [397]:
## Create a table with just the sku and the Unit cost
unit_cost = items_df[["Sku#","Unit Cost"]]
unit_cost

,Sku#,Unit Cost
0,2970,1.25
1,2974,1.10
2,2978,2.30
3,3004,1.20
4,3005,1.30


In [398]:
## Join the "unit_cost" table with "sales_df" table. Considering the sku as the primary Key. 
##This new table will be called "profit_margin"

profit_margin = pd.merge(sales_df, unit_cost, how="left", on = "Sku#")
profit_margin

,Customer Number,Customer Name,Order Number,Sku#,Version,Qty. sold,Unit Price (cost per unit),posted date,Unit Cost
0,C012345,Luis,123,2970,V11,50,10,2022-03-10,1.25
1,C012345,Luis,123,2974,V12,50,10,2022-03-10,1.10
2,C012345,Luis,123,2978,NaN,50,10,2022-03-10,2.30
3,C012345,Luis,123,3004,V1,50,10,2022-03-10,1.20
4,C012345,Luis,123,3005,V2,50,10,2022-03-10,1.30
...,...,...,...,...,...,...,...,...,...
85,C012347,Ralph,125,2970,V11,30,10,2021-10-05,1.25
86,C012347,Ralph,125,2974,V12,30,10,2021-10-05,1.10
87,C012347,Ralph,125,2978,NaN,30,10,2021-10-05,2.30
88,C012347,Ralph,125,3004,V1,30,10,2021-10-05,1.20


In [399]:
## 1.-create two extra columns "revenue" and "cost_sold".
## 2.- group the sku's by total revenue and cost sold
profit_margin["Revenue"] =  profit_margin["Qty. sold"] * profit_margin["Unit Price (cost per unit)"]
profit_margin["Cost sold"] =  profit_margin["Qty. sold"] * profit_margin["Unit Cost"]
profit_margin = profit_margin.groupby("Sku#")["Revenue","Cost sold"].sum()
profit_margin

<ipython-input-399-9c73d5e53fef>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  profit_margin = profit_margin.groupby("Sku#")["Revenue","Cost sold"].sum()


,Revenue,Cost sold
Sku#,,
2970,6300,787.5
2974,6300,693.0
2978,6300,1449.0
3004,6300,756.0
3005,6300,819.0


In [400]:
## Calculate gross profit and gross profit margen

profit_margin["Gross profit"] = profit_margin["Revenue"] - profit_margin["Cost sold"]
profit_margin["Gross profit margin"] = profit_margin["Gross profit"] / profit_margin["Revenue"]
profit_margen                                                                      

,Revenue,Cost sold,Gross profit,Gross profit margen
Sku#,,,,
2970,6300,787.500000,"$5,512.50",87.50%
2974,6300,693.000000,"$5,607.00",89.00%
2978,6300,1449.000000,"$4,851.00",77.00%
3004,6300,756.000000,"$5,544.00",88.00%
3005,6300,819.000000,"$5,481.00",87.00%


## 2.- Create dataset for dashboard

In [402]:
## this dataset will be used to create a dashboard in tableau that will display charts and important metrics

profit_margin.reset_index()
inv_df = inv_df[["Sku#","Qty. on hand"]]
dashboard_dataset = pd.merge(sales_df,profit_margin , how="left", on = "Sku#")
dashboard_dataset = pd.merge(dashboard_dataset,inv_df , how="left", on = "Sku#")
dashboard_dataset = pd.merge(dashboard_dataset,unit_cost , how="left", on = "Sku#")
dashboard_dataset["Inventory cost"] = dashboard_dataset["Qty. on hand"] * dashboard_dataset["Unit Cost"]
dashboard_dataset["Inv turns (Monthly)"] = (dashboard_dataset["Cost sold"] / dashboard_dataset["Inventory cost"])/6
dashboard_dataset["Inv turns (Yearly)"] = dashboard_dataset["Inv turns (Monthly)"]*12
dashboard_dataset = dashboard_dataset[["posted date","Sku#","Customer Name","Qty. sold","Revenue","Cost sold","Gross profit","Gross profit margin","Qty. on hand","Inventory cost","Inv turns (Monthly)","Inv turns (Yearly)"]]

dashboard_dataset


,posted date,Sku#,Customer Name,Qty. sold,Revenue,Cost sold,Gross profit,Gross profit margin,Qty. on hand,Inventory cost,Inv turns (Monthly),Inv turns (Yearly)
0,2022-03-10,2970,Luis,50,6300,787.5,5512.5,0.875,210,262.5,0.5,6.0
1,2022-03-10,2974,Luis,50,6300,693.0,5607.0,0.890,210,231.0,0.5,6.0
2,2022-03-10,2978,Luis,50,6300,1449.0,4851.0,0.770,210,483.0,0.5,6.0
3,2022-03-10,3004,Luis,50,6300,756.0,5544.0,0.880,210,252.0,0.5,6.0
4,2022-03-10,3005,Luis,50,6300,819.0,5481.0,0.870,210,273.0,0.5,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
85,2021-10-05,2970,Ralph,30,6300,787.5,5512.5,0.875,210,262.5,0.5,6.0
86,2021-10-05,2974,Ralph,30,6300,693.0,5607.0,0.890,210,231.0,0.5,6.0
87,2021-10-05,2978,Ralph,30,6300,1449.0,4851.0,0.770,210,483.0,0.5,6.0
88,2021-10-05,3004,Ralph,30,6300,756.0,5544.0,0.880,210,252.0,0.5,6.0


In [403]:
## export dataset to an excel sheet
dashboard_dataset.to_excel(r"C:\Users\jm_gv\Documents\Personal\Planning test\Dashboard\dashboard dataset.xlsx")

## 3.- Create forecast by sku

In [404]:
## here we create data set for the forecast. We will only use date, sku and units sold.
df_forecast = sales_df[["posted date","Sku#","Qty. sold"]]
df_forecast

,posted date,Sku#,Qty. sold
0,2022-03-10,2970,50
1,2022-03-10,2974,50
2,2022-03-10,2978,50
3,2022-03-10,3004,50
4,2022-03-10,3005,50
...,...,...,...
85,2021-10-05,2970,30
86,2021-10-05,2974,30
87,2021-10-05,2978,30
88,2021-10-05,3004,30


In [405]:
## add another column that displays only month and year.
df_forecast["Month"] = df_forecast['posted date'].dt.strftime('%Y-%m')
df_forecast

<ipython-input-405-2fae66abac5f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast["Month"] = df_forecast['posted date'].dt.strftime('%Y-%m')


,posted date,Sku#,Qty. sold,Month
0,2022-03-10,2970,50,2022-03
1,2022-03-10,2974,50,2022-03
2,2022-03-10,2978,50,2022-03
3,2022-03-10,3004,50,2022-03
4,2022-03-10,3005,50,2022-03
...,...,...,...,...
85,2021-10-05,2970,30,2021-10
86,2021-10-05,2974,30,2021-10
87,2021-10-05,2978,30,2021-10
88,2021-10-05,3004,30,2021-10


In [406]:
## Change the "Sku#" data type from integer to a string, if we don't do this step the code will throw an error 
## when we run the forecast
df_forecast['Sku#']=df_forecast['Sku#'].astype(str)
df_forecast

<ipython-input-406-3a934261b292>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast['Sku#']=df_forecast['Sku#'].astype(str)


,posted date,Sku#,Qty. sold,Month
0,2022-03-10,2970,50,2022-03
1,2022-03-10,2974,50,2022-03
2,2022-03-10,2978,50,2022-03
3,2022-03-10,3004,50,2022-03
4,2022-03-10,3005,50,2022-03
...,...,...,...,...
85,2021-10-05,2970,30,2021-10
86,2021-10-05,2974,30,2021-10
87,2021-10-05,2978,30,2021-10
88,2021-10-05,3004,30,2021-10


In [407]:
## create pivot table with sku's as columns and the month in the index. This format will help us to create the forecast
## in the next steps
pivot_sales = pd.pivot_table(df_forecast, columns = "Sku#", index="Month", values = "Qty. sold", aggfunc=np.sum)
pivot_sales

Sku#,2970,2974,2978,3004,3005
Month,,,,,
2021-10,105,105,105,105,105
2021-11,105,105,105,105,105
2021-12,105,105,105,105,105
2022-01,105,105,105,105,105
2022-02,105,105,105,105,105
2022-03,105,105,105,105,105


In [408]:
## here we create the forecast for every sku for the next six months.
months = ["2022-04","2022-05","2022-06","2022-07","2022-08","2022-09"]
list = ["2970","2974","2978","3004","3005"]

t =  ExponentialSmoothing(pivot_sales["2970"], trend='add').fit()
t = t.forecast(6)
t =  pd.Series(t, name="2970")
t = t.to_frame()
t["Date"] = months

b =  ExponentialSmoothing(pivot_sales["2974"], trend='add').fit()
b = b.forecast(6)
b =  pd.Series(b, name="2974")
b = b.to_frame()
b["Date"] = months
    
c = pd.merge(t, b, on = "Date")

x = 0
for sku in list:
    a =  ExponentialSmoothing(pivot_sales[sku], trend='add').fit()
    a = a.forecast(6)
    a =  pd.Series(a, name=sku)
    list[x] = a.to_frame()
    list[x]["Date"] = months
    x = x+1
    print(a)


2022-04-01    105.0
2022-05-01    105.0
2022-06-01    105.0
2022-07-01    105.0
2022-08-01    105.0
2022-09-01    105.0
Freq: MS, Name: 2970, dtype: float64
2022-04-01    105.0
2022-05-01    105.0
2022-06-01    105.0
2022-07-01    105.0
2022-08-01    105.0
2022-09-01    105.0
Freq: MS, Name: 2974, dtype: float64
2022-04-01    105.0
2022-05-01    105.0
2022-06-01    105.0
2022-07-01    105.0
2022-08-01    105.0
2022-09-01    105.0
Freq: MS, Name: 2978, dtype: float64
2022-04-01    105.0
2022-05-01    105.0
2022-06-01    105.0
2022-07-01    105.0
2022-08-01    105.0
2022-09-01    105.0
Freq: MS, Name: 3004, dtype: float64
2022-04-01    105.0
2022-05-01    105.0
2022-06-01    105.0
2022-07-01    105.0
2022-08-01    105.0
2022-09-01    105.0
Freq: MS, Name: 3005, dtype: float64


C:\Users\jm_gv\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\jm_gv\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(
C:\Users\jm_gv\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1439: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
C:\Users\jm_gv\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1444: RuntimeWarning: invalid value encountered in double_scalars
  aicc = aic + aicc_penalty
C:\Users\jm_gv\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1445: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
C:\Users\jm_gv\anaconda3\lib\site-pa

In [409]:
## Here we join every sku forecast and create a table that displays the forecast for every sku
e = pd.merge(c,list[2])
x=3
while x <5:
    #for l in list:
    e = pd.merge(e,list[x])
    x = x+1
forecast = e.set_index("Date")
forecast    


,2970,2974,2978,3004,3005
Date,,,,,
2022-04,105.0,105.0,105.0,105.0,105.0
2022-05,105.0,105.0,105.0,105.0,105.0
2022-06,105.0,105.0,105.0,105.0,105.0
2022-07,105.0,105.0,105.0,105.0,105.0
2022-08,105.0,105.0,105.0,105.0,105.0
2022-09,105.0,105.0,105.0,105.0,105.0


In [410]:
## here we added percentages and dollar signs to our profit results.
format_dict = {'Revenue':'${0:,.2f}','Cost sold':'${0:,.2f}','Gross profit':'${0:,.2f}','Cost_sold':'${0:,.2f}','Gross profit margin': '{:.2%}'}
profit_margin = profit_margin.style.format(format_dict)
profit_margin

,Revenue,Cost sold,Gross profit,Gross profit margin
Sku#,,,,
2970,"$6,300.00",$787.50,"$5,512.50",87.50%
2974,"$6,300.00",$693.00,"$5,607.00",89.00%
2978,"$6,300.00","$1,449.00","$4,851.00",77.00%
3004,"$6,300.00",$756.00,"$5,544.00",88.00%
3005,"$6,300.00",$819.00,"$5,481.00",87.00%


In [411]:
## export the forecast and the profit results to a spread sheet
with pd.ExcelWriter('outcomes.xlsx') as writer:
    profit_margin.to_excel(writer, sheet_name="profit_margin", index = "true")
    forecast.to_excel(writer, sheet_name="forecast", index = "true")
    